# Lab | Agent & Vector store

**Change the state union dataset and replicate this lab by updating the prompts accordingly.**

One such dataset is the [sonnets.txt](https://github.com/martin-gorner/tensorflow-rnn-shakespeare/blob/master/shakespeare/sonnets.txt) dataset or any other data of your choice from the same git.

# Combine agents and vector stores

This notebook covers how to combine agents and vector stores. The use case for this is that you've ingested your data into a vector store and want to interact with it in an agentic manner.

The recommended method for doing so is to create a `RetrievalQA` and then use that as a tool in the overall agent. Let's take a look at doing this below. You can do this with multiple different vector DBs, and use the agent as a way to route between them. There are two different ways of doing this - you can either let the agent use the vector stores as normal tools, or you can set `return_direct=True` to really just use the agent as a router.

## Create the vector store

In [2]:
!pip install chromadb langchain langchain_community langchain_openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 687.7 kB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 53.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 47.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 49.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.7/61.7 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 70.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 433.6/433.6 kB 29.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 85.0 MB/s eta 0:00:

In [1]:
from langchain.chains import RetrievalQA
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAI, OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader

In [2]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

In [ ]:
# If you're using colab, run this
os.environ['OPENAI_API_KEY'] ='OPENAI_API_KEY'

In [ ]:
OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

In [5]:
llm = OpenAI(temperature=0.5)

In [ ]:
from pathlib import Path

relevant_parts = []
for p in Path(".").absolute().parts:
    relevant_parts.append(p)
    if relevant_parts[-3:] == ["langchain", "docs", "modules"]:
        break
doc_path = str(Path(*relevant_parts) / "/Users/raghad/Desktop/AI-Bootcamp/Week 7/Day3/lab-agent-vector-store/state_of_the_union.txt")

In [ ]:
!pip install chromadb

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached Deprecated-1.2.18-py2.py3-none-any.whl.metadata (5.7 kB)
  Using cached importlib_metadata-8.6.1-py3-none-any.whl.metadata (4.7 kB)
  Using cached cachetools-5.5.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached pyasn1_modules-0.4.2-py3-none-any.whl.metadata (3.5 kB)
  Using cached rsa-4.9-py3-none-any.whl.metadata (4.2 kB)
  Using cached zipp-3.21.0-py3-none-any.whl.metadata (3.7 kB)
  Using cached pyasn1-0.6.1-py3-none-any.whl.metadata (8.4 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.9/16.9 MB 9.3 MB/s eta 0:00:00a 0:00:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 26.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.6/33.6 MB 15.7 MB/s eta 0:00:00a 0:00:01
Using cached Deprecated-1.2.18-py2.py3-none-any.whl (10.0 kB)
Using cached importlib_metadata-8.6.1-py3-none-any.whl (26 kB)
   ━━━━━━━━━━━━━━━━

In [8]:
loader = TextLoader(doc_path)
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

embeddings = OpenAIEmbeddings()
docsearch = Chroma.from_documents(texts, embeddings, collection_name="state-of-union")

In [9]:
state_of_union = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=docsearch.as_retriever()
)

In [10]:
from langchain_community.document_loaders import WebBaseLoader

In [11]:
loader = WebBaseLoader("https://beta.ruff.rs/docs/faq/")

In [12]:
docs = loader.load()
ruff_texts = text_splitter.split_documents(docs)
ruff_db = Chroma.from_documents(ruff_texts, embeddings, collection_name="ruff")
ruff = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=ruff_db.as_retriever()
)

## Create the Agent

In [13]:
# Import things that are needed generically
from langchain.agents import AgentType, Tool, initialize_agent
from langchain_openai import OpenAI

In [14]:
tools = [
    Tool(
        name="State of Union QA System",
        func=state_of_union.run,
        description="useful for when you need to answer questions about the most recent state of the union address. Input should be a fully formed question.",
    ),
    Tool(
        name="Ruff QA System",
        func=ruff.run,
        description="useful for when you need to answer questions about ruff (a python linter). Input should be a fully formed question.",
    ),
]

In [15]:
# Construct the agent. We will use the default agent type here.
# See documentation for a full list of options.
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

<ipython-input-15-5458be099bf6>:3: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(


In [16]:
agent.invoke(
    "What did biden say about ketanji brown jackson in the state of the union address?"
)



> Entering new AgentExecutor chain...
 I should use the State of Union QA System to answer this question since it is about the state of the union address.
Action: State of Union QA System
Action Input: "What did Biden say about Ketanji Brown Jackson in the state of the union address?"
Observation: 
Biden said that he nominated Judge Ketanji Brown Jackson for the United States Supreme Court, praising her as one of the nation's top legal minds who will continue Justice Breyer's legacy of excellence. He also mentioned that she has received broad support from both Democrats and Republicans.
Thought: I now know the final answer.
Final Answer: Biden nominated Judge Ketanji Brown Jackson for the United States Supreme Court and praised her as one of the nation's top legal minds, with broad support from both Democrats and Republicans.

> Finished chain.


{'input': 'What did biden say about ketanji brown jackson in the state of the union address?',
 'output': "Biden nominated Judge Ketanji Brown Jackson for the United States Supreme Court and praised her as one of the nation's top legal minds, with broad support from both Democrats and Republicans."}

In [17]:
agent.invoke("Why use ruff over flake8?")



> Entering new AgentExecutor chain...
 There are likely many reasons why someone might prefer to use ruff over flake8. I should gather more information to understand the differences between the two.
Action: Ruff QA System
Action Input: "What are the differences between ruff and flake8?"
Observation:  Ruff has a larger rule set and does not support custom lint rules. It also has some minor differences in its rules and does not include all the 'opinionated' rules from flake8-bugbear. Additionally, Ruff is capable of automatically fixing its own lint violations and does not support custom or third-party rules.
Thought: It seems like ruff may be more comprehensive and have more features, but it also has some limitations compared to flake8.
Action: State of Union QA System
Action Input: "What are the main features of ruff?"
Observation:  I'm sorry, I don't see any mention of "ruff" in the given context. Can you provide more information or clarify your question?
Thought: It looks like the 

{'input': 'Why use ruff over flake8?',
 'output': 'It seems like the State of Union QA System is not able to answer questions about specific linters, but it can provide information about linters in general if given enough context.'}

## Use the Agent solely as a router

You can also set `return_direct=True` if you intend to use the agent as a router and just want to directly return the result of the RetrievalQAChain.

Notice that in the above examples the agent did some extra work after querying the RetrievalQAChain. You can avoid that and just return the result directly.

In [18]:
tools = [
    Tool(
        name="State of Union QA System",
        func=state_of_union.run,
        description="useful for when you need to answer questions about the most recent state of the union address. Input should be a fully formed question.",
        return_direct=True,
    ),
    Tool(
        name="Ruff QA System",
        func=ruff.run,
        description="useful for when you need to answer questions about ruff (a python linter). Input should be a fully formed question.",
        return_direct=True,
    ),
]

In [19]:
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

In [20]:
agent.invoke(
    "What did biden say about ketanji brown jackson in the state of the union address?"
)



> Entering new AgentExecutor chain...
 I should use the State of Union QA System to answer this question.
Action: State of Union QA System
Action Input: "What did Biden say about Ketanji Brown Jackson?"
Observation:  
Biden mentioned that Ketanji Brown Jackson was nominated for the United States Supreme Court and that she is a former top litigator in private practice, a former federal public defender, and comes from a family of public school educators and police officers. He also mentioned that she is a consensus builder and has received broad support from various organizations and individuals.


> Finished chain.


{'input': 'What did biden say about ketanji brown jackson in the state of the union address?',
 'output': ' \nBiden mentioned that Ketanji Brown Jackson was nominated for the United States Supreme Court and that she is a former top litigator in private practice, a former federal public defender, and comes from a family of public school educators and police officers. He also mentioned that she is a consensus builder and has received broad support from various organizations and individuals.'}

In [21]:
agent.invoke("Why use ruff over flake8?")



> Entering new AgentExecutor chain...
 Ruff and flake8 are both python linters, so they have similar functions. However, there may be specific reasons why one is preferred over the other.
Action: Ruff QA System
Action Input: "Why use ruff over flake8?"
Observation: 
There are several reasons why someone might choose to use Ruff over Flake8:

1. More comprehensive rule set: Ruff implements over 800 rules, compared to Flake8's ~300 rules. This means that Ruff can catch a wider range of potential issues in your code.

2. Minimizes conflicts with other plugins: Ruff uses different rule codes and prefixes than other Flake8 plugins, which helps to avoid conflicts and allows for easier toggling of individual plugins.

3. Native implementation of popular plugins: Ruff natively implements popular Flake8 plugins, such as Black, isort, and yesqa, which means you don't need to install and configure these plugins separately.

4. Automatic fixing of lint violations: Unlike Flake8, Ruff is capable 

{'input': 'Why use ruff over flake8?',
 'output': "\nThere are several reasons why someone might choose to use Ruff over Flake8:\n\n1. More comprehensive rule set: Ruff implements over 800 rules, compared to Flake8's ~300 rules. This means that Ruff can catch a wider range of potential issues in your code.\n\n2. Minimizes conflicts with other plugins: Ruff uses different rule codes and prefixes than other Flake8 plugins, which helps to avoid conflicts and allows for easier toggling of individual plugins.\n\n3. Native implementation of popular plugins: Ruff natively implements popular Flake8 plugins, such as Black, isort, and yesqa, which means you don't need to install and configure these plugins separately.\n\n4. Automatic fixing of lint violations: Unlike Flake8, Ruff is capable of automatically fixing its own lint violations, making it easier to maintain code quality.\n\n5. Better compatibility with type checkers: Ruff is designed to be used alongside a type checker, and can catch s

## Multi-Hop vector store reasoning

Because vector stores are easily usable as tools in agents, it is easy to use answer multi-hop questions that depend on vector stores using the existing agent framework.

In [22]:
tools = [
    Tool(
        name="State of Union QA System",
        func=state_of_union.run,
        description="useful for when you need to answer questions about the most recent state of the union address. Input should be a fully formed question, not referencing any obscure pronouns from the conversation before.",
    ),
    Tool(
        name="Ruff QA System",
        func=ruff.run,
        description="useful for when you need to answer questions about ruff (a python linter). Input should be a fully formed question, not referencing any obscure pronouns from the conversation before.",
    ),
]

In [23]:
# Construct the agent. We will use the default agent type here.
# See documentation for a full list of options.
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

In [24]:
agent.invoke(
    "What tool does ruff use to run over Jupyter Notebooks? Did the president mention that tool in the state of the union?"
)



> Entering new AgentExecutor chain...
 You should always think about what to do
Action: Ruff QA System
Action Input: What tool does ruff use to run over Jupyter Notebooks?
Observation:  Ruff does not support source.organizeImports and source.fixAll code actions in Jupyter Notebooks.
Thought: You should always think about what to do
Action: State of Union QA System
Action Input: Did the president mention that tool in the state of the union?
Observation: 
No, the president did not mention a specific tool in the state of the union.
Thought: I now know the final answer
Final Answer: No, the president did not mention a specific tool in the state of the union.

> Finished chain.


{'input': 'What tool does ruff use to run over Jupyter Notebooks? Did the president mention that tool in the state of the union?',
 'output': 'No, the president did not mention a specific tool in the state of the union.'}